# IST 718: Big Data Analytics

- Professor: Daniel Acuna <deacuna@syr.edu>

## General instructions:

- You are welcome to discuss the problems with your classmates but __you are not allowed to copy any part of your answers either from your classmates or from the internet__
- You can put the homework files anywhere you want in your http://notebook.acuna.io workspace but _do not change_ the file names. The TAs and the professor use these names to grade your homework.
- Remove or comment out code that contains `raise NotImplementedError`. This is mainly to make the `assert` statement fail if nothing is submitted.
- The tests shown in some cells (i.e., `assert` and `np.testing.` statements) are used to grade your answers. **However, the professor and TAs will use __additional__ test for your answer. Think about cases where your code should run even if it passess all the tests you see.**
- Before downloading and submitting your work through Blackboard, remember to save and press `Validate` (or go to 
`Kernel`$\rightarrow$`Restart and Run All`). 
- Good luck!

In [12]:
# this code creates the spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
import numpy as np

# Part 1: Map-Reduce: Gradient descent

Throughout this assignment, you should use vanilla Python and not Numpy.

Some statistical models $f(x)$ are learned by optimizing a loss function $L(\Theta)$ that depends on a set of parameters $\Theta$. There are several ways of finding the optimal $\Theta$ for the loss function, one of which is to iteratively update following the gradient:

$$
\nabla L = 
\begin{pmatrix} 
    \frac{\partial L}{\partial \theta_0}\\ 
    \frac{\partial L}{\partial \theta_1} \\ 
    \vdots\\ 
    \frac{\partial L}{\partial \theta_p}
\end{pmatrix}
$$

To then, compute the update
$$\Theta^{t+1} = \Theta^t - \eta \nabla L$$

Because we assume independence between data points, the gradient becomes a summation:

$$\nabla L = \sum_{i=1}^{n} \nabla L_i$$
where $L_i$ is the loss function for the $i$-th data point.

Take as example, the statistical model $f(x) = b_0 + b_1 x$ and loss function $L(\Theta) = (f(x) - y)^2$. If we have a set of three datapoints $D=\{ (x=1,y=2), (x=-2, y=-1), (x=4, y = 3)\}$

Then the loss function for each of them is
$L_1 = \left(b_{0} + b_{1} - 2\right)^{2}$, 
$L_2 = \left(b_{0} - 2 b_{1} + 1\right)^{2}$, and
$L_3 = \left(b_{0} + 4 b_{1} - 3\right)^{2}$

with 
$$\nabla L_i = \left[\begin{matrix}2 b_{0} + 2 b_{1} x_i - 2 y_i\\2 x_i \left(b_{0} + b_{1} x_i - y_i\right)\end{matrix}\right]$$

if we start with a solution $b_0 = 0, b_1 = 1$, then the gradients are:

$$\nabla L_1 = \left[\begin{matrix}-2\\-2\end{matrix}\right]$$
$$\nabla L_2 = \left[\begin{matrix}-2\\4\end{matrix}\right]$$
$$\nabla L_3 = \left[\begin{matrix}2\\8\end{matrix}\right]$$

which after accumulation would yield
$$\nabla L = \left[\begin{matrix}-2\\10\end{matrix}\right]$$

## Question 1 (5 pts)

Create a function `f_linear(b, x)` that receives the parameters `b` and one data point `x` as lists and return the prediction for that data point. Assume that the first element of `b` is the intercept.

In [223]:
# define below the function `f_linear` which performs a linear prediction based on parameters as data point
def f_linear(b,x):
    y_hat=b[0]
    for i in range(len(x)):
        y_hat=y_hat+x[i]*b[i+1]
    return y_hat

In [224]:
# for the example above, if we assume b = [0, 1], and the first data point x = [1], y = 2
f_linear([0, 1], [1])

1

In [225]:
# test (5 pts)
assert f_linear([1, 1, 2, 3], [2, 1, 3]) == 14
assert f_linear([1], []) == 1
assert f_linear([0, 1, 0, 1, 0, 1], [0, 10, 10, 10 , 10]) == 20

## Question 2 (5 pts)
Define the function `L(y_pred, y)` that receives a prediction `y_pred` and the actual value `y` and returns the squared error between them.

In [4]:
def L(y_pred, y):
    error = (y_pred-y)*(y_pred-y)
    return error

In [5]:
# there should be not error here
L(1, 1)

0

In [6]:
# 2^2 error
L(0, 2)

4

In [7]:
# (5 pts)
assert L(1, 1) == 0
assert L(0, 4) == 16

## Question 3 (10 pts)
Create a function `gf_linear(f, b, x, y)` which returns the gradient of the linear function `f` with parameter `b` with respect to the squared loss function, evaluated at `x` and the actual outcome `y`. This function should return a vector with each element $j$ corresponding to the gradient with respect $b_j$, with $j = \{0, 1, \dots, p\}$.

In [234]:
# define `gf_linear`
def gf_linear(f, b, x, y):
    f = f_linear(b,x)
    gradient = []
    if len(x)==0:
        gradient.append(2*(f-y))
        return gradient
    else:
        gradient.append(2*(f-y))
        #y_hat = [x[i]*b[i+1] for i in range(len(x))]
        #y_hat1 = y_hat + [b[0]]
        
        #grad = [2*x[i]*(f-y) for i in range(len(x))]
        #gradient.append(grad)
        for i in range(len(x)):
            g=2*(f-y)*x[i]
            gradient.append(g)
        return gradient

In [235]:
# for the example above and first data point
x = [1]
y = 2
b = [0, 1]
gf_linear(f_linear, b, x, y)

[-2, -2]

In [236]:
# for the example above and second data point
x = [-2]
y = -1
b = [0, 1]
gf_linear(f_linear, b, x, y)

[-2, 4]

In [237]:
## (10 pts)
np.testing.assert_array_equal(gf_linear(f_linear, [0, 1], [1], 2), [-2, -2])
np.testing.assert_array_equal(gf_linear(f_linear, [0, 1], [-2], -1), [-2, 4])
np.testing.assert_array_equal(gf_linear(f_linear, [1], [], 0), [2])

## Question 4 (15 pts)

Develop a map-reduce job that produces a value so that the first element of the value is the mean loss function across all the data. You might use other pieces of information as part of the value to create your computation.

You will implement your map function as `map_mse(f, b, L, xy)` where `f` is the function `b` are the parameters of the function `L` is the loss function and `xy` is the data. Assume that the data will come as an RDD where each element is of the format:

`[x, y]` where `x` is a list and `y` is a scalar.

Since the key does not matter for this map reduce job, just put a constant of your choice.

In [226]:
# data
rdd_data = sc.parallelize([
    [[1, 2], 3],
    [[3, 1], 4],
    [[-1, 1.5], 0],
    [[-9, 3], 0]
])

In [238]:
# create function `map_mse` below
def map_mse(f, b, L, xy):
    y_hat=b[0]
    for i in range(len(xy[0])):
        y_hat=y_hat+xy[0][i]*b[i+1]
    loss = (y_hat-xy[1])*(y_hat-xy[1]) 
    return [0,[loss,1]]

You should apply the map function in the following way:

```python
# for an example set of `b = [0, 0, 0]`
rdd_data.map(lambda x: map_generator(f_linear, [0, 0, 0], L, x))
```


In [239]:
# try it here
rdd_data.map(lambda x: map_mse(f_linear, [0, 0, 0], L, x)).collect()

[[0, [9, 1]], [0, [16, 1]], [0, [0.0, 1]], [0, [0, 1]]]

In [240]:
# (10 pts)
assert rdd_data.map(lambda x: map_mse(f_linear, [0, 0, 0], L, x)).count() == 4
assert rdd_data.map(lambda x: map_mse(f_linear, [0, 0, 0], L, x)).map(lambda x: len(x)).\
    distinct().\
    first() == 2

assert rdd_data.map(lambda x: map_mse(f_linear, [0, 0, 0], L, x)).count() == 4
# the first element should be a number
assert isinstance((rdd_data.map(lambda x: map_mse(f_linear, [0, 0, 0], L, x)).first()[1][0]), 
                  (int, float, complex))
# try with other initializations
assert isinstance((rdd_data.map(lambda x: map_mse(f_linear, [1, 2, 3], L, x)).first()[1][0]), 
                  (int, float, complex))

You will now create a reduce job that receives two values of a previous reduce (or map) and merge them appropriately. Remember that at the end of the reduce job, the first element of the value should be the mean squared error. Create the function `reduce_mse(v1, v2)` below.

In [241]:
# create function `reduce_mse` below
def reduce_mse(v1, v2):
    return [((v1[0]*v1[1])+(v2[0]*v2[1]))/(v1[1]+v2[1]), 
            v1[1]+v2[1]]

In [242]:
# the following function call should return the mean squared error
rdd_data.\
    map(lambda x: map_mse(f_linear, [0, 0, 0], L, x)).\
    reduceByKey(reduce_mse).first()[1][0]

6.25

In [243]:
# the following function call should return the mean squared error
rdd_data.\
    map(lambda x: map_mse(f_linear, [2, 2, 3], L, x)).\
    reduceByKey(reduce_mse).first()[1][0]

41.8125

In [244]:
# (5 pts)
assert rdd_data.\
    map(lambda x: map_mse(f_linear, [0, 0, 0], L, x)).\
    reduceByKey(reduce_mse).first()[1][0] == 6.25

assert rdd_data.\
    map(lambda x: map_mse(f_linear, [2, 0, 0], L, x)).\
    reduceByKey(reduce_mse).first()[1][0] == 3.25

assert rdd_data.\
    map(lambda x: map_mse(f_linear, [2, 2, 3], L, x)).\
    reduceByKey(reduce_mse).first()[1][0] == 41.8125

In [245]:
rdd_data.\
    map(lambda x: map_mse(f_linear, [2, 0, 0], L, x)).\
    reduceByKey(reduce_mse).first()[1][0]

3.25

## Question 5 (10 pts)

In this question, you will compute the cumulative gradient of a model on the data. You will define a map function `map_gradient(f, gf, b, xy)` that would receive a function `f`, its gradient `gf`, its parameters `b`, and a data point `xy = [x, y]`. Also you will define a function `reduce_gradient(v1, v2)` that combines the two values appropriately. In the map function, you probably do not need to keep extra values beyond the actual gradient.

In [246]:
# define the function `map_gradient` below
def map_gradient(f, gf, b, xy):
    g=gf(f,b,xy[0],xy[1])
    return[0,g]

In [247]:
# (5 pts)
assert len(rdd_data.map(lambda xy: map_gradient(f_linear, gf_linear, [0, 0, 0], xy)[1]).first()) == 3

In [248]:
# define the function `reduce_gradient` below
def reduce_gradient(v1, v2):
    # YOUR CODE HERE
    rg=[]
    for i in range(len(v1)):
        rg.append(v1[i]+v2[i])
    return rg
    #raise NotImplementedError()

In [249]:
rdd_data.map(lambda xy: map_gradient(f_linear, gf_linear, [0, 0, 0], xy)).\
    reduceByKey(reduce_gradient).first()[1]

[-14.0, -30.0, -20.0]

In [250]:
# (5 pts)
np.testing.assert_array_equal(
    rdd_data.map(lambda xy: map_gradient(f_linear, gf_linear, [0, 0, 0], xy)).\
    reduceByKey(reduce_gradient).first()[1],
    [-14.0, -30.0, -20.0])

np.testing.assert_array_equal(
    rdd_data.map(lambda xy: map_gradient(f_linear, gf_linear, [0, 0, 0], xy)).\
    reduceByKey(reduce_gradient).first()[1],
    [-14.0, -30.0, -20.0])

if all your answers are correct, then we can run an optimization below, and the MSE should decrease with each iteration

In [251]:
b = [0, 0, 0]
learning_rate = 0.01
print("Initial solution: \t", b)
for _ in range(10):
    print("New iteration")
    print("=============")
    gradient = rdd_data.map(lambda xy: map_gradient(f_linear, gf_linear, b, xy)).\
        reduceByKey(reduce_gradient).first()[1]
    b = [b0 - learning_rate*g0 for b0, g0 in zip(b, gradient)]
    print("Current solution: \t", b)
    mse = rdd_data.\
        map(lambda x: map_mse(f_linear, b, L, x)).\
        reduceByKey(reduce_mse).first()[1][0]
    print("Current MSE: \t\t", mse)

Initial solution: 	 [0, 0, 0]
New iteration
Current solution: 	 [0.14, 0.3, 0.2]
Current MSE: 		 4.036099999999999
New iteration
Current solution: 	 [0.27480000000000004, 0.15880000000000005, 0.455]
Current MSE: 		 2.8077335025
New iteration
Current solution: 	 [0.34362200000000004, 0.41343399999999997, 0.540541]
Current MSE: 		 2.124745162297563
New iteration
Current solution: 	 [0.42466317000000003, 0.24800435, 0.707635855]
Current MSE: 		 1.7435970621414336
New iteration
Current solution: 	 [0.45430526015000006, 0.47522477825, 0.730516771125]
Current MSE: 		 1.529538732230305
New iteration
Current solution: 	 [0.50541029705925, 0.29867069991675, 0.848308677264375]
Current MSE: 		 1.4080112827808984
New iteration
Current solution: 	 [0.5135716556948637, 0.5084709260312962, 0.8371720415554381]
Current MSE: 		 1.3377595648707117
New iteration
Current solution: 	 [0.5479266281297145, 0.32798388034815074, 0.9270367149304003]
Current MSE: 		 1.2959552690942715
New iteration
Current soluti

**(5 pts)** In the code, above, play with the value of `learning_rate` less than 1.0 until the optimizer diverges (the loss function goes down and then goes *up*). What is this learning rate?

In [263]:
# YOUR CODE HERE
b = [0, 0, 0]
learning_rate = 0.009
print("Initial solution: \t", b)
for _ in range(10):
    print("New iteration")
    print("=============")
    gradient = rdd_data.map(lambda xy: map_gradient(f_linear, gf_linear, b, xy)).\
        reduceByKey(reduce_gradient).first()[1]
    b = [b0 - learning_rate*g0 for b0, g0 in zip(b, gradient)]
    print("Current solution: \t", b)
    mse = rdd_data.\
        map(lambda x: map_mse(f_linear, b, L, x)).\
        reduceByKey(reduce_mse).first()[1][0]
    print("Current MSE: \t\t", mse)

Initial solution: 	 [0, 0, 0]
New iteration
Current solution: 	 [0.112, 0.24, 0.16]
Current MSE: 		 4.234704000000001
New iteration
Current solution: 	 [0.220672, 0.197632, 0.3552]
Current MSE: 		 3.017263745024
New iteration
Current solution: 	 [0.294897664, 0.301457408, 0.470676992]
Current MSE: 		 2.267273760120193
New iteration
Current solution: 	 [0.36048288563200004, 0.30299682815999995, 0.586261239808]
Current MSE: 		 1.7999525928255933
New iteration
Current solution: 	 [0.40814832767795206, 0.35202608029696003, 0.6645021785702401]
Current MSE: 		 1.5068607387361177
New iteration
Current solution: 	 [0.44808107698664246, 0.3628787486993285, 0.7351156190122804]
Current MSE: 		 1.3223573040294068
New iteration
Current solution: 	 [0.47802637365315925, 0.3881404867532521, 0.786658238341638]
Current MSE: 		 1.2059586751193274
New iteration
Current solution: 	 [0.5022951838666727, 0.3984717197396242, 0.8307047545536558]
Current MSE: 		 1.1324247820855144
New iteration
Current solutio

Identified an optimum learning rate at **0.009**, and step towards any side results in an increase in the MSE.